In [1]:
from mpmath.tests.runtests import coverage
%load_ext autoreload
%autoreload 2

from typing import List, Optional
from omegaconf import OmegaConf
import os
import time
import json
import wandb
import logging
import argparse

import torch
import numpy as np
from datasets.driving_dataset import DrivingDataset
from datasets.my_dataset import MyDataset
from utils.misc import import_str
from models.trainers import BasicTrainer
from models.video_utils import (
    render_images,
    save_videos,
    render_novel_views
)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
cfg = OmegaConf.load(os.path.join("/mnt/e/Output/cam5/149", "config.yaml"))
%cd /home/a/drivestudio

dataset = MyDataset(cfg.data)

/home/a/drivestudio


Loading images:   4%|▍         | 8/199 [00:00<00:05, 37.13it/s]

undistorting rgb


Loading dynamic masks:   6%|▌         | 12/199 [00:00<00:03, 57.52it/s]

undistorting dynamic mask


...

Loading vehicle masks:   5%|▍         | 9/199 [00:00<00:02, 85.93it/s]

undistorting vehicle mask


Loading sky masks:   6%|▌         | 11/199 [00:00<00:01, 103.95it/s]

undistorting sky mask


Loading lidar:   0%|          | 0/199 [00:00<?, ?it/s]/home/a/drivestudio/datasets/waymo/waymo_sourceloader.py:411: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  lidar_origins = torch.from_numpy(lidar_info[:, :3]).float()  # 提取原点
Loading lidar: 100%|██████████| 199/199 [00:23<00:00,  8.33it/s]
Projecting lidar pts on images for camera front_camera: 100%|██████████| 199/199 [00:06<00:00, 33.12it/s]
Projecting lidar pts on images for camera front_left_camera: 100%|██████████| 199/199 [00:05<00:00, 33.83it/s]
Projecting lidar pts on images for camera front_right_camera: 100%|██████████| 199/199 [00:05<00:00, 33.88it/s]
Proje

In [3]:
source = dataset.lidar_source
points = source.origins + source.directions * source.ranges
grounds = source.grounds
flow_class = source.flow_classes
points.shape, grounds.shape, flow_class.shape

(torch.Size([19016548, 3]), torch.Size([19016548]), torch.Size([19016548]))

In [ ]:
import matplotlib.pyplot as plt

def plot_lidar_points(lidar_points, ground_mask=None):
    plt.figure(figsize=(10, 10))
    plt.scatter(lidar_points[:, 0][~ground_mask], lidar_points[:, 1][~ground_mask], c='b', s=1)  # 2D scatter plot
    plt.title('2D Lidar Points')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.axis('equal')
    plt.grid(True)
    plt.show()

plot_lidar_points(points.numpy(), grounds.numpy())  # Convert to numpy array for plotting


In [ ]:
import torch.nn.functional as F

def create_voxel_grid_2d(lidar_points, voxel_size=1):
    z = lidar_points[:, 2]
    lidar_points = lidar_points[(-1 <= z) & (z <= 3)]
    x_min, x_max = lidar_points[:, 0].min(), lidar_points[:, 0].max()
    y_min, y_max = lidar_points[:, 1].min(), lidar_points[:, 1].max()
    
    grid_size_x = int((x_max - x_min) / voxel_size) + 1
    grid_size_y = int((y_max - y_min) / voxel_size) + 1
    
    voxel_grid = torch.zeros((grid_size_x, grid_size_y), dtype=torch.int32)
    
    # 计算每个点的体素索引
    voxel_indices_x = ((lidar_points[:, 0] - x_min) / voxel_size).long()
    voxel_indices_y = ((lidar_points[:, 1] - y_min) / voxel_size).long()
    
    # 将二维索引展平成一维索引
    flat_indices = voxel_indices_x * grid_size_y + voxel_indices_y
    
    # 使用 torch.bincount 统计每个体素的点数量
    counts = torch.bincount(flat_indices, minlength=grid_size_x * grid_size_y)
    
    # 将一维结果重塑为二维网格
    voxel_grid = counts.view(grid_size_x, grid_size_y)
    
    return voxel_grid

p = points[~grounds]
c = flow_class[~grounds]

# 生成2D体素网格
voxel_grid_2d = create_voxel_grid_2d(p[c<=0])


In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(voxel_grid_2d.numpy() > 3, cmap='hot', interpolation='nearest')
plt.title('Voxel Grid')
plt.xlabel('Voxel X')
plt.ylabel('Voxel Y')
plt.colorbar(label='Point Count')
plt.axis('equal')
plt.show()


In [4]:

cameras = dataset.pixel_source.camera_data
camera_0 = cameras[1]

poses = camera_0.cam_to_worlds
intrinsic = camera_0.intrinsics[0]
fov = 2 * torch.atan2(intrinsic[0, 2], intrinsic[0, 0])  # 计算视场角 (FOV)

import math

pose = poses[1]     # 4x4
cx = pose[0, 3]
cy = pose[1, 3]
angle = math.atan2(pose[1, 0], pose[0, 0])
cameras



{0: <datasets.waymo.waymo_sourceloader.WaymoCameraData at 0x7f9b527cd850>,
 1: <datasets.waymo.waymo_sourceloader.WaymoCameraData at 0x7f9b8dd6a790>,
 2: <datasets.waymo.waymo_sourceloader.WaymoCameraData at 0x7f9b527d3850>,
 3: <datasets.waymo.waymo_sourceloader.WaymoCameraData at 0x7f9b527d3700>,
 4: <datasets.waymo.waymo_sourceloader.WaymoCameraData at 0x7f9b527a6f10>}

In [5]:
torch.save(cameras, 'notebook/data/cameras.pth')  # 将相机数据保存到磁盘
torch.save(points, 'notebook/data/points.pth')
torch.save(grounds, 'notebook/data/grounds.pth')
torch.save(flow_class, 'notebook/data/flow_class.pth')


In [ ]:
from utils.mytools import Camera, Grid2d, CameraSet, Grid2dNumba
grid = Grid2d(points, grounds, flow_class, cameras)




In [ ]:
grid_numba = Grid2dNumba(grid, radius=8)

In [ ]:
area_coverage = grid_numba.get_area_coverage()

In [ ]:
print(area_coverage)

In [ ]:
map = grid.get_hot_map()

map.shape, grid.voxel_grid_2d.shape

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(map.numpy(), cmap='hot', interpolation='nearest')
plt.title('Voxel Grid')
plt.xlabel('Voxel X')
plt.ylabel('Voxel Y')
plt.colorbar(label='Point Count')
plt.axis('equal')
plt.show()

In [ ]:
coverages = []
for cam in cameras.values():
    print(cam)
    coverages.append(grid.get_camera_set_coverage(CameraSet(cam.cam_to_worlds, cam.intrinsics)))
print(coverages)

In [ ]:
camera = Camera(pose, intrinsic)
grid.get_camera_coverage(camera)